In [2]:
import numpy as np 
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from scipy import interpolate
from scipy import signal
import time
import matplotlib.pyplot as plt

In [3]:
def readWISDM(usuario):
    pasta = "C:\\Users\\Usuario\\Desktop\\wisdm-dataset\\wisdm-dataset\\raw\\phone\\"
    fileA = 'data_' + str(usuario) + '_accel_phone.txt'
    fileG = 'data_' + str(usuario) + '_gyro_phone.txt'
    acc = np.loadtxt(pasta+'accel\\'+fileA, delimiter=',', comments=';', unpack=True,
                     dtype={'names': ('Subject-id', 'Activity Label', 'Timestamp', 'x', 'y', 'z'),
                            'formats': ('i4', 'U1', 'u8', 'f8', 'f8', 'f8')})
    gyr = np.loadtxt(pasta+'gyro\\'+fileG, delimiter=',', comments=';', unpack=True,
                     dtype={'names': ('Subject-id', 'Activity Label', 'Timestamp', 'x', 'y', 'z'),
                            'formats': ('i4', 'U1', 'u8', 'f8', 'f8', 'f8')})
    return acc, gyr

In [4]:
acc, gyr = readWISDM(1649)
labels = np.unique(acc[1])
labels

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'O', 'P', 'Q', 'R', 'S'], dtype='<U1')

In [5]:
rotulos = np.array(['walking', 'jogging', 'stairs', 'sitting', 'standing'])
conversao = [2, 5, 6, 0, 1]
# Rótulos WISDM
# 0: walking 
# 1: jogging 
# 2: stairs
# 3: sitting
# 4: standing
# Rotulos padronizados
# 0: Sit
# 1: Stand
# 2: Walk 
# 3: Upstairs
# 4: Downstairs
# 5: Run/Jogging
# 6: Stairs

In [7]:
chunk = np.array()
cont = 0

mfi1 = []
vfi1 = []
mfi2 = []
vfi2 = []
for usuario in tqdm(range(1600,1651)): #1600
    acc, gyr = readWISDM(usuario)
    A = np.array(acc[3:])
    G = np.array(gyr[3:])
    for atividade in range(5): #18):
        ind1 = acc[1]==labels[atividade]
        ind2 = gyr[1]==labels[atividade]
        if ~np.all(~ind1) and ~np.all(~ind2):
            t1 = acc[2][ind1]/1000000000
            t2 = gyr[2][ind2]/1000000000
            t1 = t1-t1[0]
            t2 = t2-t2[0]
            
            ### Removendo janelas
            if np.any(np.diff(t1)<0):
                pos = np.nonzero(np.diff(t1)<0)[0].astype(int)
                for k in pos:
                    t1[k+1:] = t1[k+1:]+t1[k]+1/20
            if np.any(np.diff(t2)<0):
                pos = np.nonzero(np.diff(t2)<0)[0].astype(int)
                for k in pos:
                    t2[k+1:] = t2[k+1:]+t2[k]+1/20

            fA = A[:,ind1]
            fG = G[:,ind2]

            ### Interpolando
            intp1 = interpolate.interp1d(t1, fA, kind='cubic')
            intp2 = interpolate.interp1d(t2, fG, kind='cubic')
            nt1 = np.arange(0,t1[-1],1/20)
            nt2 = np.arange(0,t2[-1],1/20)
            fA = intp1(nt1)
            fG = intp2(nt2)

            
            # mfi1.append(np.mean(1/np.diff(t1)))
            # vfi1.append(np.std(1/np.diff(t1)))
            # mfi2.append(np.mean(1/np.diff(t2)))
            # vfi2.append(np.std(1/np.diff(t2)))

            NJ = min(fA.shape[1]//60, fG.shape[1]//60)
            aux = np.zeros(60*6+3)
            for i in range(NJ):
                aux[:180] = fA[:,i*60:(i+1)*60].reshape(60*3)
                aux[180:360] = fG[:,i*60:(i+1)*60].reshape(60*3)
                if atividade < 5:
                    aux[-1] = conversao[atividade]
                aux[-2] = usuario
                aux[-3] = atividade
                chunk.append(aux)

  0%|          | 0/51 [00:00<?, ?it/s]

In [14]:
comgravidade = chunk[:15196,:]

array([[-3.64761350e-01, -9.00995470e-01,  1.98847712e+00, ...,
         0.00000000e+00,  1.60000000e+03,  2.00000000e+00],
       [ 8.10018945e-02, -5.95542451e-02,  4.29259811e-01, ...,
         0.00000000e+00,  1.60000000e+03,  2.00000000e+00],
       [ 1.92588077e-01, -8.55499056e-01, -1.24274639e+00, ...,
         0.00000000e+00,  1.60000000e+03,  2.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=comgravidade[1310,:360], mode='lines'))

In [ ]:
px.histogram(comgravidade[:,-3])

In [ ]:
fig = px.scatter(y=mfi1, error_y=vfi1)
fig.add_trace(go.Scatter(y=mfi2, 
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=vfi2,
            visible=True), mode='markers', showlegend=False))
fig.show()

In [ ]:
colunas = ['accel-x-' + str(x) for x in np.arange(60).tolist()]
colunas += ['accel-y-' + str(x) for x in np.arange(60).tolist()]
colunas += ['accel-z-' + str(x) for x in np.arange(60).tolist()]
colunas += ['gyro-x-' + str(x) for x in np.arange(60).tolist()]
colunas += ['gyro-y-' + str(x) for x in np.arange(60).tolist()]
colunas += ['gyro-z-' + str(x) for x in np.arange(60).tolist()]
colunas += ['activity code']
colunas += ['user']
colunas += ['standard activity code']
colunas

In [ ]:
dfcg = pd.DataFrame(data=comgravidade)
dfcg.columns = colunas
dfcg = dfcg.astype({"activity code": int, "user": int, "standard activity code": int})

In [ ]:
treinocg = dfcg[dfcg['user']<=1635]
testecg = dfcg[dfcg['user']>1635]
treinocg.shape[0]/(treinocg.shape[0]+testecg.shape[0])

In [ ]:
folder = 'C:\\Users\\Usuario\\Desktop\\data\\WISDM\\original_view\\'
sub = 'WISDM_balanced_20Hz_with_gravity\\'
treinocg.to_csv(folder+'train.csv',index=False)
testecg.to_csv(folder+'test.csv',index=False)

In [ ]:
folder = 'C:\\Users\\Narayana\\Desktop\\organizando datasets\\WISDM organizado\\'
df0 = pd.read_csv(folder+'WISDM_V0.csv')
treino = pd.read_csv(folder+'WISDM_V2_train.csv')
teste = pd.read_csv(folder+'WISDM_V2_test.csv')

In [ ]:
print('Fração de treino e teste', '\nTreino: ', treino.shape[0]/(treino.shape[0]+teste.shape[0]),
      '\nTeste: ', teste.shape[0]/(treino.shape[0]+teste.shape[0]))

In [ ]:
treino.to_csv('WISDM_V2_train.csv',index=False)
teste.to_csv('WISDM_V2_test.csv',index=False)

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import umap